In [4]:
!pip install natasha

Defaulting to user installation because normal site-packages is not writeable
Using legacy setup.py install for intervaltree, since package 'wheel' is not installed.
    Running setup.py install for intervaltree: started
    Running setup.py install for intervaltree: finished with status 'done'


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


In [5]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [59]:
import re

*Открываем русский текст (с омонимами)*

In [67]:
with open('rus.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [68]:
cleant = re.compile('<.*?>')
cleantext = re.sub(cleant, ' ', text)
cleanre = re.compile('[^а-яёА-ЯЁ ]')
cleantext = re.sub(cleanre, ' ', text)
print(cleantext)

Ты белых лебедей кормила откинув тяжесть черных кос  Я рядом плыл  сошлись кормила  закатный луч был странно кос  Вдруг лебедей метнулась пара  Не знаю  чья была вина  Закат замлел за дымкой пара алея  как поток вина  Мгновенья двигались и стали  лишь ты паришь  свой свет струя  Меж тем в реке   из сизой стали влачится за струей струя  Важно белый корабль  с тысячью  если и не больше  аккуратных одинаковых окошек плыл по воздуху  разрезая холодную хрустальную гладь парков  Весна пришла  Роскошь тонкого стекла фоторамка в виде ананаса   настоящее спасение от плохого настроения   Полковник едет в кузове машины  с ним рядом фельдшер  И тут впереди оказывается запевала  Мелодия крепнет  Она требует марша 


*Парсим в pymorphy*

In [69]:
def tokenizing(text):
    text = text.lower()
    ls = []
    token = word_tokenize(text)
    for word in token:
        an = morph.parse(word)
        optag = an[0].tag
        pos = str(optag).split(',')
        ls.append(pos[0])
    return(ls)

In [315]:
pym = tokenizing(cleantext)
for i in pym:
    if i == 'NPRO':
        i = 'PRON'
    elif i == 'GRND':
        i = 'VERB'
    elif i == 'ADJF':
        i = 'ADJ'
    elif i == 'PREP':
        i = 'ADP'
print(pym)

['NPRO', 'NOUN', 'NOUN', 'VERB', 'GRND', 'NOUN', 'NOUN', 'NOUN', 'NPRO', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADJF', 'NOUN', 'VERB', 'ADVB', 'NOUN', 'ADVB', 'NOUN', 'VERB', 'NOUN', 'PRCL', 'VERB', 'ADJF', 'VERB', 'NOUN', 'NOUN', 'VERB', 'PREP', 'NOUN', 'NOUN', 'GRND', 'CONJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'CONJ', 'VERB', 'PRCL', 'NPRO', 'VERB', 'ADJF', 'NOUN', 'GRND', 'PREP', 'CONJ', 'PREP', 'NOUN', 'PREP', 'ADJF', 'VERB', 'VERB', 'PREP', 'NOUN', 'GRND', 'ADJS', 'ADJF', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'CONJ', 'PRCL', 'COMP', 'ADJF', 'ADJF', 'NOUN', 'VERB', 'PREP', 'NOUN', 'GRND', 'ADJF', 'ADJF', 'NOUN', 'ADJS', 'NOUN', 'ADJS femn', 'NOUN', 'ADJF', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PREP', 'ADJF', 'NOUN', 'NOUN', 'VERB', 'PREP', 'NOUN', 'NOUN', 'PREP', 'NPRO', 'NOUN', 'NOUN', 'CONJ', 'ADVB', 'ADVB', 'VERB', 'VERB', 'NOUN', 'VERB', 'NPRO', 'VERB', 'NOUN']


*Парсим в mystem*

In [72]:
from pymystem3 import Mystem

In [250]:
def myst(text):
    m = Mystem()
    ms = m.analyze(cleantext)
    gr = []
    l = []
    for i in ms:
        if (i.get('text') != ' ') and (i.get('text') != ' \n') and (i.get('text') != '  ') and (i.get('text') != '   '):
            l.append(i)
    for k in l:
        ana = k.get('analysis')
        grpos = ana[0].get('gr')
        mys = grpos.split(',')
        gr.append(mys[0])
    return(gr)

In [320]:
mys = myst(cleantext)


*Парсим в Natasha*

In [342]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
doc = Doc(cleantext)

In [343]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
sent = doc.sents[0].morph
nat = []
rez = []
for j in sent:
    nat.append(j)
n = list(nat[0])
for i in n:
    rez.append(list(i)[1])
print(rez)

['PRON', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'PRON', 'ADV', 'VERB', 'VERB', 'VERB', 'ADJ', 'NOUN', 'AUX', 'ADV', 'ADJ', 'ADV', 'NOUN', 'VERB', 'NOUN', 'PART', 'VERB', 'DET', 'AUX', 'NOUN', 'PROPN', 'VERB', 'ADP', 'NOUN', 'NOUN', 'VERB', 'SCONJ', 'NOUN', 'NOUN', 'PROPN', 'VERB', 'CCONJ', 'VERB', 'PART', 'PRON', 'VERB', 'DET', 'NOUN', 'NOUN', 'PROPN', 'PRON', 'ADP', 'NOUN', 'ADP', 'NOUN', 'VERB', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADV', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'SCONJ', 'PART', 'PART', 'NUM', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'ADP', 'NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'NOUN', 'PROPN', 'VERB', 'PROPN', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PROPN', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADP', 'PRON', 'ADV', 'NOUN', 'CCONJ', 'ADV', 'ADV', 'VERB', 'VERB', 'PROPN', 'VERB', 'PRON', 'VERB', 'NOUN']


In [336]:
from sklearn.metrics import accuracy_score

In [347]:
correct = ['PRON', 'ADJ', 'NOUN', 'VERB' , 'ADJ' , 'NOUN' , 'ADJ', 'NOUN', 'PRON', 'ADV', 'VERB' , 'VERB' , 'VERB' ,  'ADJ' , 'NOUN' , 'VERB' , 'ADV' , 'ADJ' , 'ADV' , 'NOUN' , 'VERB', 'NOUN', 'PART', 'VERB' , 'DET' , 'VERB' , 'NOUN' , 'NOUN' , 'VERB' , 'ADP' , 'NOUN' , 'NOUN' , 'VERB' , 'CONJ' , 'NOUN' , 'NOUN' , 'NOUN' , 'VERB' , 'CONJ' , 'VERB' , 'PART' , 'PRON' , 'VERB' , 'DET' , 'NOUN' , 'NOUN' , 'ADP' , 'PRON' ,'ADP', 'NOUN', 'ADP', 'NOUN' ,'VERB', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADV', 'ADJ', 'NOUN', 'ADP', 'NOUN' , 'CONJ' , 'PART', 'PART', 'NUM', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'ADP', 'NOUN', 'VERB', 'ADJ', 'ADJ' , 'NOUN' , 'NOUN' , 'NOUN' , 'VERB' , 'NOUN' , 'ADJ', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'ADJ', 'NOUN' , 'ADP', 'ADJ', 'NOUN', 'PROPN', 'VERB', 'ADP', 'NOUN', 'NOUN', 'ADP', 'PRON' 'ADV' , 'NOUN' , 'CONJ' ,'ADV', 'ADV', 'VERB', 'NOUN' , 'NOUN' , 'VERB', 'PRON', 'VERB', 'NOUN']
accuracy_score(rez, correct)

ValueError: Found input variables with inconsistent numbers of samples: [110, 109]

In [332]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Program Files\Python37\python.exe -m pip install --upgrade pip' command.


In [323]:
import spacy

*Открываем английский текст с омонимами*

In [324]:
with open('eng.txt', 'r', encoding='utf-8') as f:
    eng = f.read()

In [329]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(eng)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [301]:
! pip install flair

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


ERROR: Could not find a version that satisfies the requirement torch>=1.1.0 (from flair) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch>=1.1.0 (from flair)
You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.
